In [1]:
MISTRAL_API_KEY = "323c4f0f-16d1-41c8-858e-a041f13dc964"

In [2]:
# ============================================================
# Mistral AI 기반 정책뉴스 요약 + 키워드 추출
# ============================================================

!pip -q install pandas requests tqdm

import pandas as pd
import requests
import json
from tqdm import tqdm

# ============================================================
# 설정
# ============================================================

MISTRAL_API_KEY = "여기에_본인_API_KEY"
MODEL_NAME = "mistral-small-latest"  # 안정적 + 빠름

INPUT_CSV  = "/mnt/data/policy_news_20251223_020451.csv"
OUTPUT_CSV = "policy_news_mistral_summary.csv"

# ============================================================
# Mistral API 호출 함수
# ============================================================

def call_mistral(prompt):
    url = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "너는 한국 정부 정책뉴스를 요약하는 전문가야."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3
    }

    res = requests.post(url, headers=headers, data=json.dumps(payload), timeout=60)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# ============================================================
# 데이터 로드
# ============================================================

df = pd.read_csv(INPUT_CSV)
print("총 레코드 수:", len(df))

# ============================================================
# 요약 + 키워드 추출
# ============================================================

summaries = []
keywords_list = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    content = str(row["DataContents"])[:4000]  # 토큰 안전장치

    prompt = f"""
아래 정책뉴스 본문을 보고 다음을 수행해줘.

1. 핵심 내용을 한국어로 정확히 **2줄 요약**
2. 정책 관련 **핵심 키워드 5개**를 쉼표로 구분

출력 형식은 반드시 아래처럼 해줘.

요약:
- 문장1
- 문장2

키워드:
키워드1, 키워드2, 키워드3, 키워드4, 키워드5

본문:
{content}
"""

    try:
        result = call_mistral(prompt)

        # 결과 파싱
        summary_match = result.split("키워드:")
        summary_text = summary_match[0].replace("요약:", "").strip()
        keyword_text = summary_match[1].strip() if len(summary_match) > 1 else ""

        summaries.append(summary_text)
        keywords_list.append(keyword_text)

    except Exception as e:
        summaries.append("")
        keywords_list.append("")
        print("Error:", e)

# ============================================================
# 결과 저장
# ============================================================

df["AI_Summary"] = summaries
df["AI_Keywords"] = keywords_list

df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("\n✅ 완료")
print("저장 파일:", OUTPUT_CSV)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/policy_news_20251223_020451.csv'

In [4]:
# ============================================================
# Mistral AI 정책뉴스 요약 + 키워드 추출 (안정 버전)
# ============================================================

!pip -q install pandas requests tqdm

import pandas as pd
import requests
import json
import time
from tqdm import tqdm

# ============================================================
# 설정
# ============================================================

MISTRAL_API_KEY = "323c4f0f-16d1-41c8-858e-a041f13dc964"
MODEL_NAME = "mistral-small-latest"

INPUT_CSV  = "/mnt/data/policy_news_20251223_020451.csv"
OUTPUT_CSV = "policy_news_mistral_summary.csv"

# ============================================================
# CSV 로드 확인
# ============================================================

df = pd.read_csv(INPUT_CSV)
print("📄 로드된 레코드 수:", len(df))

# ============================================================
# Mistral API 호출 함수 (에러 방어)
# ============================================================

def call_mistral(prompt):
    url = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "너는 한국 정부 정책뉴스 요약 전문가야."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2
    }

    res = requests.post(url, headers=headers, json=payload, timeout=60)

    # HTTP 에러 출력
    if res.status_code != 200:
        raise Exception(f"HTTP {res.status_code}: {res.text}")

    data = res.json()

    # 응답 구조 방어
    if "choices" not in data or len(data["choices"]) == 0:
        raise Exception("Empty response from Mistral")

    return data["choices"][0]["message"]["content"]

# ============================================================
# 요약 + 키워드 처리
# ============================================================

summaries = []
keywords = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = str(row.get("DataContents", ""))[:3500]

        prompt = f"""
아래 정책뉴스를 분석해줘.

[작업]
1. 핵심 내용을 **2문장 요약**
2. 정책 관련 **핵심 키워드 5개 추출**

[출력 형식]
요약:
- 문장1
- 문장2

키워드:
키워드1, 키워드2, 키워드3, 키워드4, 키워드5

[본문]
{text}
"""

        result = call_mistral(prompt)

        # ---------- 결과 파싱 ----------
        summary = ""
        keyword = ""

        if "키워드:" in result:
            parts = result.split("키워드:")
            summary = parts[0].replace("요약:", "").strip()
            keyword = parts[1].strip()
        else:
            summary = result.strip()

        summaries.append(summary)
        keywords.append(keyword)

        time.sleep(0.8)  # rate limit 방어

    except Exception as e:
        summaries.append("")
        keywords.append("")
        print(f"⚠️ 오류 (row {idx}):", e)
        time.sleep(1.5)

# ============================================================
# 결과 저장
# ============================================================

df["AI_Summary"] = summaries
df["AI_Keywords"] = keywords

df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("\n✅ 완료!")
print("📁 저장 파일:", OUTPUT_CSV)


📄 로드된 레코드 수: 50


  0%|          | 0/50 [00:00<?, ?it/s]

⚠️ 오류 (row 0): HTTP 401: {"detail":"Unauthorized"}


  2%|▏         | 1/50 [00:01<01:23,  1.71s/it]

⚠️ 오류 (row 1): HTTP 401: {"detail":"Unauthorized"}


...

⚠️ 오류 (row 19): HTTP 401: {"detail":"Unauthorized"}


 40%|████      | 20/50 [00:34<00:51,  1.70s/it]

⚠️ 오류 (row 20): HTTP 401: {"detail":"Unauthorized"}


 40%|████      | 20/50 [00:34<00:52,  1.74s/it]


KeyboardInterrupt: 

In [7]:
import requests

API_KEY = "eJX6BRdA9UllO3f1I6PbcDnz9ZUn7uCc"

res = requests.get(
    "https://api.mistral.ai/v1/models",
    headers={"Authorization": f"Bearer {API_KEY}"}
)

print(res.status_code)
print(res.text)


200
{"object":"list","data":[{"id":"mistral-medium-2505","object":"model","created":1766456412,"owned_by":"mistralai","capabilities":{"completion_chat":true,"function_calling":true,"completion_fim":false,"fine_tuning":true,"vision":true,"ocr":false,"classification":false,"moderation":false,"audio":false},"name":"mistral-medium-2505","description":"Our frontier-class multimodal model released May 2025.","max_context_length":131072,"aliases":[],"deprecation":null,"deprecation_replacement_model":null,"default_model_temperature":0.3,"type":"base"},{"id":"mistral-medium-2508","object":"model","created":1766456412,"owned_by":"mistralai","capabilities":{"completion_chat":true,"function_calling":true,"completion_fim":false,"fine_tuning":true,"vision":true,"ocr":false,"classification":false,"moderation":false,"audio":false},"name":"mistral-medium-2508","description":"Update on Mistral Medium 3 with improved capabilities.","max_context_length":131072,"aliases":["mistral-medium-latest","mistral-m

In [8]:
# ============================================================
# Mistral AI 최신 API (responses) - 정책뉴스 요약
# ============================================================

!pip -q install pandas requests tqdm

import pandas as pd
import requests
import json
import time
from tqdm import tqdm

# ===============================
# 설정
# ===============================

MISTRAL_API_KEY = "eJX6BRdA9UllO3f1I6PbcDnz9ZUn7uCc"
MODEL_NAME = "mistral-small-latest"

INPUT_CSV  = "/mnt/data/policy_news_20251223_020451.csv"
OUTPUT_CSV = "policy_news_mistral_summary.csv"

# ===============================
# API 호출 함수 (responses)
# ===============================

def call_mistral(text):
    url = "https://api.mistral.ai/v1/responses"

    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "input": f"""
아래 정책뉴스를 분석해줘.

1. 핵심 내용을 한국어로 2문장 요약
2. 정책 관련 핵심 키워드 5개 추출

형식:
요약:
- 문장1
- 문장2

키워드:
키워드1, 키워드2, 키워드3, 키워드4, 키워드5

본문:
{text}
"""
    }

    res = requests.post(url, headers=headers, json=payload, timeout=60)

    if res.status_code != 200:
        raise Exception(res.text)

    data = res.json()
    return data["output"][0]["content"][0]["text"]

# ===============================
# 데이터 로드
# ===============================

df = pd.read_csv(INPUT_CSV)
print("📄 레코드 수:", len(df))

summaries = []
keywords = []

# ===============================
# 처리 루프
# ===============================

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = str(row["DataContents"])[:3500]
        result = call_mistral(text)

        if "키워드:" in result:
            s, k = result.split("키워드:")
            summaries.append(s.replace("요약:", "").strip())
            keywords.append(k.strip())
        else:
            summaries.append(result.strip())
            keywords.append("")

        time.sleep(0.7)

    except Exception as e:
        print(f"❌ row {idx} 실패:", e)
        summaries.append("")
        keywords.append("")
        time.sleep(1.5)

# ===============================
# 저장
# ===============================

df["AI_Summary"] = summaries
df["AI_Keywords"] = keywords
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("✅ 완료:", OUTPUT_CSV)


📄 레코드 수: 50


  0%|          | 0/50 [00:00<?, ?it/s]

❌ row 0 실패: {
  "message":"no Route matched with those values",
  "request_id":"2aa60d337aa5db62be875e466c794814"
}


  2%|▏         | 1/50 [00:01<01:35,  1.95s/it]

❌ row 1 실패: {
  "message":"no Route matched with those values",
  "request_id":"df8e502c0da4746086c8354091da2d18"
}


...

 32%|███▏      | 16/50 [00:27<00:57,  1.70s/it]

❌ row 16 실패: {
  "message":"no Route matched with those values",
  "request_id":"36a22c447a66461bd206b36d96e7000a"
}


 34%|███▍      | 17/50 [00:29<00:57,  1.73s/it]

❌ row 17 실패: {
  "message":"no Route matched with those values",
  "request_id":"c4252a445917f66899b013515b7cdb0d"
}


KeyboardInterrupt: 

In [9]:
# ============================================================
# 1. 라이브러리
# ============================================================
!pip install pandas requests tqdm

import pandas as pd
import requests
import time
from tqdm import tqdm

# ============================================================
# 2. 설정
# ============================================================
CSV_PATH = "/mnt/data/policy_news_20251223_020451.csv"

MISTRAL_API_KEY = "eJX6BRdA9UllO3f1I6PbcDnz9ZUn7uCc"

API_URL = "https://api.mistral.ai/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# ============================================================
# 3. Mistral 호출 함수 (정상 엔드포인트)
# ============================================================
def call_mistral(text):
    payload = {
        "model": "mistral-small-latest",
        "messages": [
            {
                "role": "system",
                "content": "너는 한국어 정책 뉴스 요약 전문가야."
            },
            {
                "role": "user",
                "content": f"""
아래 정책 뉴스 내용을 참고해서:

1️⃣ 핵심 요약 2줄
2️⃣ 주요 키워드 5개 (콤마로 구분)

뉴스 내용:
{text}
"""
            }
        ],
        "temperature": 0.3
    }

    res = requests.post(API_URL, headers=HEADERS, json=payload, timeout=60)

    if res.status_code != 200:
        raise Exception(f"HTTP {res.status_code}: {res.text}")

    return res.json()["choices"][0]["message"]["content"]

# ============================================================
# 4. CSV 로드
# ============================================================
df = pd.read_csv(CSV_PATH)
print(f"📄 레코드 수: {len(df)}")

# ============================================================
# 5. 요약 실행
# ============================================================
summaries = []
keywords = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = str(row["DataContents"])[:3500]
        result = call_mistral(text)

        # 결과 분리
        parts = result.split("키워드")
        summary = parts[0].strip()
        keyword = parts[1].strip() if len(parts) > 1 else ""

        summaries.append(summary)
        keywords.append(keyword)

        time.sleep(1.2)  # 과도한 요청 방지

    except Exception as e:
        print(f"❌ row {idx} 실패:", e)
        summaries.append("")
        keywords.append("")
        time.sleep(1.5)

# ============================================================
# 6. 결과 저장
# ============================================================
df["summary_2lines"] = summaries
df["keywords"] = keywords

OUTPUT_PATH = "/mnt/data/policy_news_mistral_summary.csv"
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

print("✅ 완료:", OUTPUT_PATH)


📄 레코드 수: 50


100%|██████████| 50/50 [02:07<00:00,  2.54s/it]

✅ 완료: /mnt/data/policy_news_mistral_summary.csv


In [10]:
첨부한 파일의 내용을 mistral ai 에 보내서,보고서 초안 생성하는 google colab 에서 동작하는 python 코드 작성해줘

SyntaxError: invalid syntax (ipython-input-1035148067.py, line 1)

In [11]:
# ============================================================
# CSV → Mistral AI → Markdown 보고서 초안 생성
# Google Colab 실행용 (최신 Mistral API 규격)
# ============================================================

!pip -q install pandas requests tqdm

import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime

# =========================
# 1. 설정
# =========================

CSV_PATH = "/mnt/data/policy_news_mistral_summary.csv"
OUTPUT_MD = "policy_ai_report_draft.md"

MISTRAL_API_KEY = "eJX6BRdA9UllO3f1I6PbcDnz9ZUn7uCc"

MODEL = "mistral-large-latest"
API_URL = "https://api.mistral.ai/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# =========================
# 2. CSV 로드
# =========================

df = pd.read_csv(CSV_PATH)
print(f"📄 로드된 레코드 수: {len(df)}")

# 사용할 컬럼 자동 선택
text_cols = [c for c in df.columns if "summary" in c.lower() or "content" in c.lower()]
if not text_cols:
    raise ValueError("❌ 요약/본문 컬럼을 찾을 수 없습니다.")

# 전체 텍스트 결합 (너무 길면 자동 절단)
combined_text = "\n\n".join(
    df[text_cols].astype(str).apply(lambda x: " ".join(x), axis=1)
)

combined_text = combined_text[:12000]  # 토큰 안전 제한

# =========================
# 3. Mistral 프롬프트
# =========================

prompt = f"""
다음은 대한민국 정책 뉴스 요약 데이터입니다.

이 데이터를 기반으로
📌 정책 보고서 초안(Markdown)을 작성하세요.

요구사항:
- 제목 포함
- Executive Summary
- AI 관련 정책 동향
- 주요 정책 사례 정리 (불릿)
- 시사점 및 향후 과제
- 공공 보고서 톤 (객관적, 공식적)

데이터:
\"\"\"
{combined_text}
\"\"\"
"""

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "너는 대한민국 정책 보고서를 작성하는 전문 분석가다."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

# =========================
# 4. Mistral 호출
# =========================

print("🚀 Mistral AI 보고서 생성 중...")

res = requests.post(API_URL, headers=HEADERS, json=payload)

if res.status_code != 200:
    raise Exception(f"❌ Mistral API 오류: {res.status_code}\n{res.text}")

result = res.json()["choices"][0]["message"]["content"]

# =========================
# 5. Markdown 저장
# =========================

with open(OUTPUT_MD, "w", encoding="utf-8") as f:
    f.write(result)

print("✅ 보고서 초안 생성 완료!")
print("📄 파일:", OUTPUT_MD)


📄 로드된 레코드 수: 50
🚀 Mistral AI 보고서 생성 중...
✅ 보고서 초안 생성 완료!
📄 파일: policy_ai_report_draft.md


In [14]:
# ===============================
# 1. 라이브러리 설치
# ===============================
!pip install markdown

# ===============================
# 2. 기본 설정
# ===============================
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import markdown

# 👉 수정 필요 부분
#GMAIL_USER = "보내는계정@gmail.com"
#GMAIL_APP_PASSWORD = "앱비밀번호16자리"
#TO_EMAIL = "받는사람@gmail.com"

MD_FILE_PATH = "/content/policy_ai_report_draft.md"

# ===============================
# 3. Markdown 파일 읽기
# ===============================
with open(MD_FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

# ===============================
# 4. Markdown → HTML 변환
# ===============================
html_body = markdown.markdown(
    md_text,
    extensions=["extra", "tables", "fenced_code"]
)

# ===============================
# 5. HTML 메일 포맷
# ===============================
html_template = f"""
<html>
<head>
  <meta charset="UTF-8">
</head>
<body style="font-family: Arial, sans-serif; line-height:1.6;">
{html_body}
</body>
</html>
"""

# ===============================
# 6. 이메일 생성
# ===============================
msg = MIMEMultipart("alternative")
msg["Subject"] = "📄 AI 정책 보고서 초안"
msg["From"] = GMAIL_USER
msg["To"] = TO_EMAIL

msg.attach(MIMEText(html_template, "html", "utf-8"))

# ===============================
# 7. Gmail 전송
# ===============================
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
server.send_message(msg)
server.quit()

print("✅ Gmail 본문으로 Markdown 보고서 전송 완료!")


UnicodeEncodeError: 'ascii' codec can't encode characters in position 1-5: ordinal not in range(128)

In [20]:
# @title
# ===============================
# 1. 라이브러리 설치
# ===============================
!pip install markdown

from google.colab import userdata
userdata.get('secretName')
# ===============================
# 2. 라이브러리 로드
# ===============================
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import markdown

# ===============================
# 3. Markdown 파일 경로
# ===============================
MD_FILE_PATH = "/content/policy_ai_report_draft.md"

# ===============================
# 4. Markdown 파일 읽기
# ===============================
with open(MD_FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

# ===============================
# 5. Markdown → HTML 변환
# ===============================
html_body = markdown.markdown(
    md_text,
    extensions=["extra", "tables", "fenced_code"]
)

# ===============================
# 6. HTML 메일 포맷
# ===============================
html_template = f"""
<html>
<head>
  <meta charset="UTF-8">
</head>
<body style="font-family: Arial, sans-serif; line-height:1.6;">
{html_body}
</body>
</html>
"""

# ===============================
# 7. 이메일 메시지 생성
# ===============================
msg = MIMEMultipart("alternative")
msg["Subject"] = "📄 AI 정책 보고서 초안"
msg["From"] = GMAIL_USER
msg["To"] = TO_EMAIL

msg.attach(MIMEText(html_template, "html", "utf-8"))

# ===============================
# 8. Gmail SMTP 전송
# ===============================
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
server.send_message(msg)
server.quit()

print("✅ Gmail 본문으로 Markdown 보고서 전송 완료!")


UnicodeEncodeError: 'ascii' codec can't encode characters in position 1-5: ordinal not in range(128)

In [19]:
from google.colab import userdata
userdata.get('secretName')

SecretNotFoundError: Secret secretName does not exist.

In [21]:
# ===============================
# 1. 라이브러리 설치
# ===============================
!pip install markdown

# ===============================
# 2. 기본 설정
# ===============================
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import markdown

# 👉 수정 필요 부분
GMAIL_USER = "mtslg1207@gmail.com"
GMAIL_APP_PASSWORD = "gwaz iapz ratu chxb"
TO_EMAIL = "mtslg1207@gmail.com"

MD_FILE_PATH = "/content/policy_ai_report_draft.md"

# ===============================
# 3. Markdown 파일 읽기
# ===============================
with open(MD_FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

# ===============================
# 4. Markdown → HTML 변환
# ===============================
html_body = markdown.markdown(
    md_text,
    extensions=["extra", "tables", "fenced_code"]
)

# ===============================
# 5. HTML 메일 포맷
# ===============================
html_template = f"""
<html>
<head>
  <meta charset="UTF-8">
</head>
<body style="font-family: Arial, sans-serif; line-height:1.6;">
{html_body}
</body>
</html>
"""

# ===============================
# 6. 이메일 생성
# ===============================
msg = MIMEMultipart("alternative")
msg["Subject"] = "📄 AI 정책 보고서 초안"
msg["From"] = GMAIL_USER
msg["To"] = TO_EMAIL

msg.attach(MIMEText(html_template, "html", "utf-8"))

# ===============================
# 7. Gmail 전송
# ===============================
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
server.send_message(msg)
server.quit()

print("✅ Gmail 본문으로 Markdown 보고서 전송 완료!")


✅ Gmail 본문으로 Markdown 보고서 전송 완료!


In [24]:
# =========================================================
# Markdown 파일 → Gmail 본문 HTML 전송 (Colab 1셀)
# =========================================================

!pip -q install markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import markdown

# =========================
# 1. 사용자 설정
# =========================

GMAIL_ID = "mtslg1207@gmail.com"
GMAIL_APP_PASSWORD = "gwaz iapz ratu chxb"
TO_EMAIL = "mtslg1207@gmail.com"

MD_FILE_PATH = "/content/policy_ai_report_draft.md"
EMAIL_SUBJECT = "[정책 보고서] AI 정책 동향 분석 보고서"

# =========================
# 2. Markdown → HTML
# =========================

with open(MD_FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

html_body = markdown.markdown(
    md_text,
    extensions=["extra", "tables", "toc"]
)

# HTML 이메일 기본 스타일
html_template = f"""
<html>
<head>
<meta charset="utf-8">
<style>
body {{
    font-family: Arial, 'Malgun Gothic', sans-serif;
    line-height: 1.6;
}}
h1, h2, h3 {{
    color: #1a237e;
}}
ul {{
    margin-left: 20px;
}}
</style>
</head>
<body>
{html_body}
</body>
</html>
"""

# =========================
# 3. 이메일 구성
# =========================

msg = MIMEMultipart("alternative")
msg["From"] = GMAIL_ID
msg["To"] = TO_EMAIL
msg["Subject"] = EMAIL_SUBJECT

msg.attach(MIMEText(html_template, "html", "utf-8"))

# =========================
# 4. Gmail 전송
# =========================

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(GMAIL_ID, GMAIL_APP_PASSWORD)
    server.send_message(msg)

print("✅ Gmail로 Markdown 보고서 본문 전송 완료!")


✅ Gmail로 Markdown 보고서 본문 전송 완료!


In [25]:
# =========================================================
# Markdown → Gmail HTML 본문 (Gmail 호환 최적화)
# =========================================================

!pip -q install markdown

import smtplib
from email.mime.text import MIMEText
import markdown

# =========================
# 1. 설정
# =========================

GMAIL_ID = "mtslg1207@gmail.com"
GMAIL_APP_PASSWORD = "gwaz iapz ratu chxb"
TO_EMAIL = "mtslg1207@gmail.com"

MD_FILE_PATH = "/content/policy_ai_report_draft.md"
EMAIL_SUBJECT = "[정책 보고서] AI 정책 동향 분석 보고서"

# =========================
# 2. Markdown → HTML
# =========================

with open(MD_FILE_PATH, "r", encoding="utf-8") as f:
    md_text = f.read()

html_body = markdown.markdown(
    md_text,
    extensions=["extra", "tables"]
)

# Gmail 친화적 inline 스타일
html_body = html_body.replace("<h1>", "<h1 style='color:#1a237e;'>")
html_body = html_body.replace("<h2>", "<h2 style='color:#303f9f;'>")
html_body = html_body.replace("<h3>", "<h3 style='color:#3949ab;'>")
html_body = html_body.replace("<p>", "<p style='line-height:1.6;font-size:14px;'>")
html_body = html_body.replace("<ul>", "<ul style='margin-left:20px;'>")

html_content = f"""
<html>
<body style="font-family:Arial,'Malgun Gothic',sans-serif;">
{html_body}
</body>
</html>
"""

# =========================
# 3. Gmail 전송 (HTML ONLY)
# =========================

msg = MIMEText(html_content, "html", "utf-8")
msg["From"] = GMAIL_ID
msg["To"] = TO_EMAIL
msg["Subject"] = EMAIL_SUBJECT

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(GMAIL_ID, GMAIL_APP_PASSWORD)
    server.send_message(msg)

print("✅ Markdown → HTML 본문 메일 전송 완료!")


✅ Markdown → HTML 본문 메일 전송 완료!
